# RNN and LSTM Evaluation
This notebook will evaluate trained RNN and LSTM models for the overal test loss and accuracy and class specific test loss and accuracy. It will load in the saved model files and the preprocessed data files to do the evaluation.

In [1]:
import numpy as np
import os
import pickle
import random
import tensorflow as tf
from dropconnect_tensorflow import DropConnect

## Initialization

In [2]:
seed = 22
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
tf.keras.utils.set_random_seed(seed)

In [3]:
save_path = './Saved Models'

In [4]:
weight_decays_l1 = [0.001, 0.01, 0.1]
weight_decays_l2 = [0.001, 0.01, 0.1]
dropout_rates = [0.1, 0.2, 0.3]
dropconnect_rates = [0.1, 0.2, 0.3]

In [8]:
def get_metrics(X, y, parameters, batch_size, save_path, model_type, regularization_type, dataset):
    model = tf.keras.models.load_model(f'{save_path}/{model_type}_l1_weight_decay_0_{dataset}.h5', custom_objects={'DropConnect': DropConnect})
    metrics = model.evaluate(X, y, batch_size=batch_size)
    losses = [metrics[0]]
    accuracies = [metrics[1]]
    for parameter in parameters:
        model_file = f'{save_path}/{model_type}_{regularization_type}_{parameter}_{dataset}.h5'
        model = tf.keras.models.load_model(model_file, custom_objects={'DropConnect': DropConnect})
        metrics = model.evaluate(X, y, batch_size=batch_size)
        losses.append(metrics[0])
        accuracies.append(metrics[1])
    return [losses, accuracies]

In [9]:
def get_class_metrics(X, y, parameters, batch_size, save_path, model_type, regularization_type, dataset):
    num_classes = len(set(y))
    metrics = get_metrics(X, y, parameters, batch_size, save_path, model_type, regularization_type, dataset)
    print('total', metrics)
    class_metrics = {'total': metrics}
    for class_index in range(num_classes):
        indices = np.where(y == class_index)[0]
        X_class = X[indices]
        y_class = y[indices]
        metrics = get_metrics(X_class, y_class, parameters, batch_size, save_path, model_type, regularization_type, dataset)
        class_metrics[class_index] = metrics
        print(class_index, metrics)
    with open(f'./Results/{model_type}_{regularization_type}_{dataset}.pickle', 'wb') as f:
        pickle.dump(class_metrics, f)
    return class_metrics

## Patent Classification Dataset Evaluation

In [5]:
with open('./Temp Data/patent_classification.pickle', 'rb') as f:
    data = pickle.load(f)
X, y = data
X_test, y_test = X[2], y[2]

In [25]:
class_metrics = get_class_metrics(X_test, y_test, weight_decays_l1, 128, save_path, 'rnn', 'l1_weight_decay', 'patent_classification')

32/32 [==============================] - 1s 28ms/step - loss: 2.4065 - accuracy: 0.2892
total [[2.4184155464172363, 1.6751713752746582, 1.7022242546081543, 2.406501054763794], [0.49900001287460327, 0.515500009059906, 0.4230000078678131, 0.289249986410141]]
3/3 [==============================] - 0s 30ms/step - loss: 3.4474 - accuracy: 0.0346
0 [[2.3104941844940186, 2.3879144191741943, 1.925428032875061, 3.4474027156829834], [0.5561959743499756, 0.5072045922279358, 0.3775216042995453, 0.03458213433623314]]
7/7 [==============================] - 1s 28ms/step - loss: 0.9875 - accuracy: 0.9035
1 [[1.8937536478042603, 0.7870141267776489, 0.8086224794387817, 0.9875468015670776], [0.5743473172187805, 0.7196367979049683, 0.7480136156082153, 0.9035187363624573]]
1/1 [==============================] - 0s 285ms/step - loss: 4.4728 - accuracy: 0.0000e+00
2 [[4.3528923988342285, 3.6786234378814697, 3.2108030319213867, 4.472818374633789], [0.08737864345312119, 0.0, 0.0, 0.0]]
4/4 [===================

In [26]:
class_metrics = get_class_metrics(X_test, y_test, weight_decays_l2, 128, save_path, 'rnn', 'l2_weight_decay', 'patent_classification')

32/32 [==============================] - 1s 29ms/step - loss: 1.5992 - accuracy: 0.4235
total [[2.4184155464172363, 2.1715526580810547, 1.9862427711486816, 1.5991626977920532], [0.49900001287460327, 0.5052499771118164, 0.5120000243186951, 0.4235000014305115]]
3/3 [==============================] - 1s 68ms/step - loss: 1.7250 - accuracy: 0.3602
0 [[2.3104941844940186, 1.930533528327942, 1.909980297088623, 1.7250266075134277], [0.5561959743499756, 0.590778112411499, 0.619596540927887, 0.36023053526878357]]
7/7 [==============================] - 0s 29ms/step - loss: 1.0032 - accuracy: 0.5392
1 [[1.8937536478042603, 1.7941431999206543, 1.3298189640045166, 1.003159523010254], [0.5743473172187805, 0.6129398345947266, 0.6038592457771301, 0.5391600728034973]]
1/1 [==============================] - 0s 286ms/step - loss: 3.1251 - accuracy: 0.0000e+00
2 [[4.3528923988342285, 3.5788414478302, 3.7604856491088867, 3.125063896179199], [0.08737864345312119, 0.20388349890708923, 0.10679611563682556, 0.

In [27]:
class_metrics = get_class_metrics(X_test, y_test, dropout_rates, 128, save_path, 'rnn', 'dropout', 'patent_classification')

32/32 [==============================] - 1s 29ms/step - loss: 1.8783 - accuracy: 0.5130
total [[2.4184155464172363, 2.0115580558776855, 1.8363388776779175, 1.8782602548599243], [0.49900001287460327, 0.5034999847412109, 0.5082499980926514, 0.5130000114440918]]
3/3 [==============================] - 0s 29ms/step - loss: 1.2523 - accuracy: 0.7205
0 [[2.3104941844940186, 1.832220196723938, 1.6895347833633423, 1.2523425817489624], [0.5561959743499756, 0.619596540927887, 0.6340057849884033, 0.7204610705375671]]
7/7 [==============================] - 0s 30ms/step - loss: 0.9350 - accuracy: 0.7367
1 [[1.8937536478042603, 1.2541742324829102, 0.8985904455184937, 0.93503338098526], [0.5743473172187805, 0.660612940788269, 0.7139614224433899, 0.7366628646850586]]
1/1 [==============================] - 0s 286ms/step - loss: 3.9923 - accuracy: 0.1262
2 [[4.3528923988342285, 4.587277412414551, 3.527890920639038, 3.992339849472046], [0.08737864345312119, 0.058252427726984024, 0.09708737581968307, 0.126

In [28]:
class_metrics = get_class_metrics(X_test, y_test, dropconnect_rates, 128, save_path, 'rnn', 'dropconnect', 'patent_classification')

32/32 [==============================] - 1s 28ms/step - loss: 2.2215 - accuracy: 0.4970
total [[2.4184155464172363, 2.640918254852295, 2.1304848194122314, 2.221475124359131], [0.49900001287460327, 0.4872500002384186, 0.5074999928474426, 0.4970000088214874]]
3/3 [==============================] - 0s 29ms/step - loss: 2.0128 - accuracy: 0.5908
0 [[2.3104941844940186, 2.2777159214019775, 2.274653911590576, 2.0127909183502197], [0.5561959743499756, 0.5648415088653564, 0.5417867302894592, 0.590778112411499]]
7/7 [==============================] - 0s 29ms/step - loss: 1.9091 - accuracy: 0.5743
1 [[1.8937536478042603, 3.052103042602539, 0.988713264465332, 1.90913724899292], [0.5743473172187805, 0.44154369831085205, 0.7253121733665466, 0.5743473172187805]]
1/1 [==============================] - 0s 293ms/step - loss: 4.7533 - accuracy: 0.1262
2 [[4.3528923988342285, 5.348640441894531, 5.6102800369262695, 4.753262996673584], [0.08737864345312119, 0.1650485396385193, 0.09708737581968307, 0.126213

In [24]:
class_metrics = get_class_metrics(X_test, y_test, weight_decays_l1, 64, save_path, 'lstm', 'l1_weight_decay', 'patent_classification')

63/63 [==============================] - 2s 12ms/step - loss: 3.5496 - accuracy: 0.2103
total [[2.245410680770874, 1.6233084201812744, 2.130558490753174, 3.549597978591919], [0.5297499895095825, 0.5170000195503235, 0.21025000512599945, 0.21025000512599945]]
6/6 [==============================] - 1s 12ms/step - loss: 3.9738 - accuracy: 0.0000e+00
0 [[2.3159451484680176, 1.5691262483596802, 2.55407977104187, 3.9738337993621826], [0.6109510064125061, 0.6080691814422607, 0.0, 0.0]]
14/14 [==============================] - 1s 12ms/step - loss: 3.1874 - accuracy: 0.0000e+00
1 [[1.6595555543899536, 1.0927199125289917, 1.7678930759429932, 3.1874046325683594], [0.6367763876914978, 0.6787741184234619, 0.0, 0.0]]
2/2 [==============================] - 1s 14ms/step - loss: 5.0264 - accuracy: 0.0000e+00
2 [[5.127684593200684, 3.133763551712036, 3.6001598834991455, 5.026421546936035], [0.3106796145439148, 0.0, 0.0, 0.0]]
7/7 [==============================] - 1s 12ms/step - loss: 3.8626 - accuracy: 

In [21]:
class_metrics = get_class_metrics(X_test, y_test, weight_decays_l2, 64, save_path, 'lstm', 'l2_weight_decay', 'patent_classification')

63/63 [==============================] - 1s 12ms/step - loss: 1.5400 - accuracy: 0.4945
total [[2.245410680770874, 2.2225375175476074, 1.6371864080429077, 1.5400022268295288], [0.5297499895095825, 0.515999972820282, 0.515999972820282, 0.4945000112056732]]
6/6 [==============================] - 1s 12ms/step - loss: 1.3345 - accuracy: 0.6455
0 [[2.3159451484680176, 2.4310874938964844, 1.8951497077941895, 1.3345122337341309], [0.6109510064125061, 0.559077799320221, 0.5677233338356018, 0.6455331444740295]]
14/14 [==============================] - 1s 12ms/step - loss: 1.2433 - accuracy: 0.5051
1 [[1.6595555543899536, 1.3472672700881958, 1.2205675840377808, 1.243330717086792], [0.6367763876914978, 0.6719636917114258, 0.6186152100563049, 0.5051078200340271]]
2/2 [==============================] - 1s 14ms/step - loss: 3.0390 - accuracy: 0.0000e+00
2 [[5.127684593200684, 5.473664283752441, 2.912576913833618, 3.0389585494995117], [0.3106796145439148, 0.20388349890708923, 0.019417475908994675, 0.

In [22]:
class_metrics = get_class_metrics(X_test, y_test, dropout_rates, 64, save_path, 'lstm', 'dropout', 'patent_classification')

63/63 [==============================] - 2s 12ms/step - loss: 1.9034 - accuracy: 0.5322
total [[2.245410680770874, 2.0409154891967773, 2.007172107696533, 1.9033937454223633], [0.5297499895095825, 0.527999997138977, 0.5360000133514404, 0.5322499871253967]]
6/6 [==============================] - 1s 13ms/step - loss: 1.8746 - accuracy: 0.6427
0 [[2.3159451484680176, 2.197852849960327, 2.5385847091674805, 1.874626874923706], [0.6109510064125061, 0.579250693321228, 0.5619596838951111, 0.6426513195037842]]
14/14 [==============================] - 1s 12ms/step - loss: 1.2455 - accuracy: 0.6720
1 [[1.6595555543899536, 1.2376114130020142, 1.243467926979065, 1.245504379272461], [0.6367763876914978, 0.695800244808197, 0.6799091696739197, 0.6719636917114258]]
2/2 [==============================] - 1s 13ms/step - loss: 4.0467 - accuracy: 0.2330
2 [[5.127684593200684, 4.323151111602783, 4.163130283355713, 4.046717166900635], [0.3106796145439148, 0.291262149810791, 0.33980581164360046, 0.233009710907

In [23]:
class_metrics = get_class_metrics(X_test, y_test, dropconnect_rates, 64, save_path, 'lstm', 'dropconnect', 'patent_classification')

63/63 [==============================] - 2s 12ms/step - loss: 2.2472 - accuracy: 0.5238
total [[2.245410680770874, 2.4130122661590576, 2.151954174041748, 2.2471611499786377], [0.5297499895095825, 0.5177500247955322, 0.5292500257492065, 0.5237500071525574]]
6/6 [==============================] - 1s 12ms/step - loss: 2.2566 - accuracy: 0.5504
0 [[2.3159451484680176, 2.6230850219726562, 2.083460807800293, 2.256643533706665], [0.6109510064125061, 0.5619596838951111, 0.6167147159576416, 0.5504322648048401]]
14/14 [==============================] - 1s 12ms/step - loss: 1.5064 - accuracy: 0.6595
1 [[1.6595555543899536, 1.5837503671646118, 1.4290080070495605, 1.5063905715942383], [0.6367763876914978, 0.657207727432251, 0.6674233675003052, 0.6594778895378113]]
2/2 [==============================] - 1s 14ms/step - loss: 4.5886 - accuracy: 0.3398
2 [[5.127684593200684, 5.301477909088135, 4.776759624481201, 4.588621139526367], [0.3106796145439148, 0.3009708821773529, 0.26213592290878296, 0.3398058

## Bankings77 Dataset Evaluation

In [6]:
with open('./Temp Data/bankings_77.pickle', 'rb') as f:
    data = pickle.load(f)
X, y = data
X_test, y_test = X[2], y[2]

In [12]:
class_metrics = get_class_metrics(X_test, y_test, weight_decays_l1, 128, save_path, 'rnn', 'l1_weight_decay', 'bankings_77')

13/13 [==============================] - 1s 28ms/step - loss: 36.1299 - accuracy: 0.6675
total [[1.55243718624115, 1.49589204788208, 8.22491455078125, 36.1298942565918], [0.6737499833106995, 0.7537500262260437, 0.6931250095367432, 0.6675000190734863]]
1/1 [==============================] - 0s 282ms/step - loss: 35.6959 - accuracy: 0.7407
0 [[1.0757124423980713, 1.300416111946106, 7.647181034088135, 35.695858001708984], [0.5925925970077515, 0.7407407164573669, 0.7407407164573669, 0.7407407164573669]]
1/1 [==============================] - 0s 287ms/step - loss: 35.7777 - accuracy: 0.8333
1 [[1.1793688535690308, 0.9976730942726135, 7.17648458480835, 35.77769470214844], [0.875, 0.8333333134651184, 0.8333333134651184, 0.8333333134651184]]
1/1 [==============================] - 0s 304ms/step - loss: 35.3658 - accuracy: 0.7647
2 [[4.033124923706055, 5.108354568481445, 9.796774864196777, 35.36582946777344], [0.6470588445663452, 0.9411764740943909, 0.7647058963775635, 0.7647058963775635]]
1/1 [

In [13]:
class_metrics = get_class_metrics(X_test, y_test, weight_decays_l2, 128, save_path, 'rnn', 'l2_weight_decay', 'bankings_77')

13/13 [==============================] - 1s 28ms/step - loss: 2.6018 - accuracy: 0.6831
total [[1.55243718624115, 1.561215877532959, 1.5368592739105225, 2.6018240451812744], [0.6737499833106995, 0.7281249761581421, 0.734375, 0.6831250190734863]]
1/1 [==============================] - 0s 297ms/step - loss: 2.2943 - accuracy: 0.8519
0 [[1.0757124423980713, 1.6842867136001587, 1.4573321342468262, 2.2943406105041504], [0.5925925970077515, 0.7777777910232544, 0.7777777910232544, 0.8518518805503845]]
1/1 [==============================] - 0s 286ms/step - loss: 2.4044 - accuracy: 0.7917
1 [[1.1793688535690308, 0.9291748404502869, 1.0782840251922607, 2.404449939727783], [0.875, 0.8333333134651184, 0.875, 0.7916666865348816]]
1/1 [==============================] - 0s 287ms/step - loss: 2.7233 - accuracy: 0.8235
2 [[4.033124923706055, 2.382983684539795, 2.2578253746032715, 2.7232980728149414], [0.6470588445663452, 0.9411764740943909, 0.8823529481887817, 0.8235294222831726]]
1/1 [================

In [14]:
class_metrics = get_class_metrics(X_test, y_test, dropout_rates, 128, save_path, 'rnn', 'dropout', 'bankings_77')

13/13 [==============================] - 1s 28ms/step - loss: 1.1411 - accuracy: 0.7613
total [[1.55243718624115, 1.258467435836792, 1.2609540224075317, 1.1411216259002686], [0.6737499833106995, 0.734375, 0.7443749904632568, 0.7612500190734863]]
1/1 [==============================] - 0s 288ms/step - loss: 0.7374 - accuracy: 0.7778
0 [[1.0757124423980713, 1.1086785793304443, 1.4826451539993286, 0.7374073266983032], [0.5925925970077515, 0.7407407164573669, 0.7037037014961243, 0.7777777910232544]]
1/1 [==============================] - 0s 292ms/step - loss: 0.7836 - accuracy: 0.8750
1 [[1.1793688535690308, 0.5375629663467407, 0.37571975588798523, 0.7836140990257263], [0.875, 0.875, 0.875, 0.875]]
1/1 [==============================] - 0s 285ms/step - loss: 2.0329 - accuracy: 0.9412
2 [[4.033124923706055, 2.8073079586029053, 0.4676800072193146, 2.0329294204711914], [0.6470588445663452, 0.8823529481887817, 0.8823529481887817, 0.9411764740943909]]
1/1 [==============================] - 1s 58

In [15]:
class_metrics = get_class_metrics(X_test, y_test, dropconnect_rates, 128, save_path, 'rnn', 'dropconnect', 'bankings_77')

13/13 [==============================] - 1s 28ms/step - loss: 1.6023 - accuracy: 0.7169
total [[1.55243718624115, 1.5100457668304443, 1.3571051359176636, 1.6022937297821045], [0.6737499833106995, 0.7431250214576721, 0.6768749952316284, 0.7168750166893005]]
1/1 [==============================] - 0s 278ms/step - loss: 1.4771 - accuracy: 0.7037
0 [[1.0757124423980713, 1.0594887733459473, 1.3636466264724731, 1.4771268367767334], [0.5925925970077515, 0.7407407164573669, 0.7407407164573669, 0.7037037014961243]]
1/1 [==============================] - 0s 295ms/step - loss: 1.2653 - accuracy: 0.7917
1 [[1.1793688535690308, 0.42291393876075745, 1.1709249019622803, 1.265260934829712], [0.875, 0.8333333134651184, 0.8333333134651184, 0.7916666865348816]]
1/1 [==============================] - 0s 316ms/step - loss: 4.9918 - accuracy: 0.8235
2 [[4.033124923706055, 4.474112510681152, 1.3498278856277466, 4.991763591766357], [0.6470588445663452, 0.9411764740943909, 0.8823529481887817, 0.8235294222831726

In [31]:
class_metrics = get_class_metrics(X_test, y_test, weight_decays_l1, 64, save_path, 'lstm', 'l1_weight_decay', 'bankings_77')

25/25 [==============================] - 1s 8ms/step - loss: 4.7227 - accuracy: 0.0162
total [[1.8968113660812378, 1.7245216369628906, 3.338845729827881, 4.722686290740967], [0.746874988079071, 0.7406250238418579, 0.09812500327825546, 0.016249999403953552]]
1/1 [==============================] - 1s 709ms/step - loss: 4.5128 - accuracy: 0.0000e+00
0 [[0.7608888149261475, 2.5705292224884033, 2.589975118637085, 4.5127716064453125], [0.8888888955116272, 0.7407407164573669, 0.0, 0.0]]
1/1 [==============================] - 1s 690ms/step - loss: 4.6863 - accuracy: 0.0000e+00
1 [[2.606218099594116, 1.7492116689682007, 3.20575213432312, 4.686312675476074], [0.875, 0.9166666865348816, 0.0, 0.0]]
1/1 [==============================] - 1s 696ms/step - loss: 4.7969 - accuracy: 0.0000e+00
2 [[2.138291358947754, 2.1428415775299072, 2.7485501766204834, 4.796919822692871], [0.8235294222831726, 0.8235294222831726, 0.6470588445663452, 0.0]]
1/1 [==============================] - 1s 705ms/step - loss: 4.

In [32]:
class_metrics = get_class_metrics(X_test, y_test, weight_decays_l2, 64, save_path, 'lstm', 'l2_weight_decay', 'bankings_77')

25/25 [==============================] - 1s 9ms/step - loss: 1.5424 - accuracy: 0.6844
total [[1.8968113660812378, 1.6996554136276245, 1.6394578218460083, 1.5424467325210571], [0.746874988079071, 0.7325000166893005, 0.7356250286102295, 0.684374988079071]]
1/1 [==============================] - 1s 663ms/step - loss: 1.1926 - accuracy: 0.8148
0 [[0.7608888149261475, 2.191287040710449, 0.9918436408042908, 1.1926318407058716], [0.8888888955116272, 0.7407407164573669, 0.8148148059844971, 0.8148148059844971]]
1/1 [==============================] - 1s 705ms/step - loss: 1.3416 - accuracy: 0.8333
1 [[2.606218099594116, 0.870078980922699, 1.1202540397644043, 1.3416045904159546], [0.875, 0.8333333134651184, 0.7916666865348816, 0.8333333134651184]]
1/1 [==============================] - 1s 680ms/step - loss: 3.0199 - accuracy: 0.8824
2 [[2.138291358947754, 2.1487271785736084, 2.5007483959198, 3.0199356079101562], [0.8235294222831726, 0.9411764740943909, 0.9411764740943909, 0.8823529481887817]]
1/

In [10]:
class_metrics = get_class_metrics(X_test, y_test, dropout_rates, 64, save_path, 'lstm', 'dropout', 'bankings_77')

2022-05-08 01:53:16.546094: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-08 01:53:16.557784: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-08 01:53:16.558463: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-08 01:53:16.559678: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

25/25 [==============================] - 1s 9ms/step - loss: 1.1497 - accuracy: 0.7937
total [[1.8968113660812378, 1.520137906074524, 1.387440800666809, 1.1497468948364258], [0.746874988079071, 0.78125, 0.7943750023841858, 0.793749988079071]]
1/1 [==============================] - 1s 665ms/step - loss: 0.7652 - accuracy: 0.7778
0 [[0.7608888149261475, 1.3530776500701904, 1.0369895696640015, 0.7652258276939392], [0.8888888955116272, 0.8888888955116272, 0.7777777910232544, 0.7777777910232544]]
1/1 [==============================] - 1s 665ms/step - loss: 0.5837 - accuracy: 0.9167
1 [[2.606218099594116, 1.0169744491577148, 1.4336185455322266, 0.5836617350578308], [0.875, 0.8333333134651184, 0.875, 0.9166666865348816]]
1/1 [==============================] - 1s 668ms/step - loss: 1.2723 - accuracy: 0.8824
2 [[2.138291358947754, 2.201218366622925, 1.5811262130737305, 1.2723355293273926], [0.8235294222831726, 0.9411764740943909, 0.9411764740943909, 0.8823529481887817]]
1/1 [===================

In [11]:
class_metrics = get_class_metrics(X_test, y_test, dropconnect_rates, 64, save_path, 'lstm', 'dropconnect', 'bankings_77')

25/25 [==============================] - 1s 8ms/step - loss: 1.8152 - accuracy: 0.7631
total [[1.8968113660812378, 1.5662028789520264, 1.7353307008743286, 1.8151535987854004], [0.746874988079071, 0.7787500023841858, 0.7649999856948853, 0.7631250023841858]]
1/1 [==============================] - 1s 663ms/step - loss: 0.8868 - accuracy: 0.8519
0 [[0.7608888149261475, 0.9092178344726562, 1.2581349611282349, 0.8868033289909363], [0.8888888955116272, 0.8148148059844971, 0.7777777910232544, 0.8518518805503845]]
1/1 [==============================] - 1s 675ms/step - loss: 1.5176 - accuracy: 0.8750
1 [[2.606218099594116, 2.0574817657470703, 0.9209091663360596, 1.5176138877868652], [0.875, 0.875, 0.875, 0.875]]
1/1 [==============================] - 1s 659ms/step - loss: 1.3354 - accuracy: 0.9412
2 [[2.138291358947754, 1.9612747430801392, 2.636693000793457, 1.3354254961013794], [0.8235294222831726, 0.8823529481887817, 0.8823529481887817, 0.9411764740943909]]
1/1 [==============================]